<a href="https://colab.research.google.com/github/SoumadeepMandal/Smart_chatbot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display

In [ ]:
!pip install nltk

In [ ]:
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.5 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=d4805030ebd29f8d5e35d89ec65c4944ff84d8ac0e158d477c0f0182313747b6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=d09b1ce91bc25cb6b6353c2bc2203604db235d67ac00ed4f4849faa21a7cbbe5
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1ec

In [ ]:
!pip install lxml_html_clean


from newspaper import Article
import random
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('punkt',quiet=True)

True

In [ ]:
try:
    nltk.data.find('tokenizers/punkt_tab/english/')
except LookupError:
    nltk.download('punkt_tab', quiet=True)

In [ ]:
article = Article('https://my.clevelandclinic.org/health/diseases/15096-chronic-kidney-disease')
article.download()
article.parse()
article.nlp()
corpus=article.text

In [ ]:
print(corpus)

What is chronic kidney disease?

Chronic kidney disease (CKD and chronic renal disease) means that there’s damage to your kidneys and they aren’t working as well as they should. Your kidneys are like a filter in your body — filtering out wastes, toxins and extra water from your blood. They also help with other functions like bone and red blood cell health. When your kidneys begin to lose their function, they can’t filter waste, which means the waste builds up in your blood.

Advertisement Cleveland Clinic is a non-profit academic medical center. Advertising on our site helps support our mission. We do not endorse non-Cleveland Clinic products or services. Policy

Kidney disease is called “chronic” because kidney function slowly decreases over time. CKD can lead to kidney failure, which is also called end-stage kidney disease. Not everyone with CKD will develop kidney failure, but the disease will often worsen without treatment. There’s no cure for chronic kidney disease. But there are 

In [ ]:
text = corpus   #tokenization
sentence_list=nltk.sent_tokenize(text)# a list of sentence

In [ ]:
print(sentence_list)  #print the list of sentence

['What is chronic kidney disease?', 'Chronic kidney disease (CKD and chronic renal disease) means that there’s damage to your kidneys and they aren’t working as well as they should.', 'Your kidneys are like a filter in your body — filtering out wastes, toxins and extra water from your blood.', 'They also help with other functions like bone and red blood cell health.', 'When your kidneys begin to lose their function, they can’t filter waste, which means the waste builds up in your blood.', 'Advertisement Cleveland Clinic is a non-profit academic medical center.', 'Advertising on our site helps support our mission.', 'We do not endorse non-Cleveland Clinic products or services.', 'Policy\n\nKidney disease is called “chronic” because kidney function slowly decreases over time.', 'CKD can lead to kidney failure, which is also called end-stage kidney disease.', 'Not everyone with CKD will develop kidney failure, but the disease will often worsen without treatment.', 'There’s no cure for chr

In [ ]:
# a function to return a random gretting  to the user
def greetting_response(text):
    text=text.lower()
    bot_greetings =['howdy' ,'hello', 'hi', 'hey','hola']
    User_greetings =['hi','hey','hello','whatssup']
    for word in text.split():
        if word in User_greetings:
            return random.choice(bot_greetings)


In [ ]:
def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))

    x = list_var
    for i in range(length):
        for j in range(length):
          if x[list_index[i]]> x[list_index[j]]:
            temp = list_index[i]
            list_index[i] = list_index[j]
            list_index[j] = temp
    return list_index

In [ ]:
#bot response
def bot_response(user_input):
    user_input = user_input.lower()
    sentence_list.append(user_input)
    bot_response = ''
    cm = CountVectorizer().fit_transform(sentence_list)
    similarity_scores = cosine_similarity(cm[-1], cm)
    similarity_scores_list = similarity_scores.flatten()
    index = index_sort(similarity_scores_list)

    # Corrected the slicing syntax to get all indices except the first one
    index = index[1:]  # Get all elements from the second element onwards

    response_flag = 0

    j = 0
    # Looping through the sliced index list
    for i in range(len(index)):
        # Check if the similarity score for the current index is greater than 0
        if similarity_scores_list[index[i]] > 0.0:
            # Append the corresponding sentence to the bot response
            bot_response = bot_response + ' ' + sentence_list[index[i]]
            response_flag = 1
            j = j + 1
            # If we have added 3 sentences, break the loop
            if j > 2:
                break

    # If no relevant sentences were found (response_flag is still 0)
    if response_flag == 0:
        bot_response = bot_response + ' ' + "I apologize, I dont understand"

    # Remove the user input sentence from the list before returning the response
    sentence_list.remove(user_input)

    return bot_response

In [ ]:
#start the chat
print('Doc  : hey I am your doc today! ask me anything')
exit_list = ['exit', 'byee','see you again', 'break']
while(True):
    user_input = input()
    if user_input.lower() in exit_list:
        print('Doc : Bye Bye take care')
        break
    else:
        if greetting_response(user_input) != None:
            print('Doc : '+greetting_response(user_input))
        else:
            print('Doc : '+bot_response(user_input))



Doc  : hey I am your doc today! ask me anything
what is kidney problems
Doc :  What is chronic kidney disease? Policy

Kidney disease is called “chronic” because kidney function slowly decreases over time. CKD can lead to kidney failure, which is also called end-stage kidney disease.
exit
Doc : Bye Bye take care
